In [10]:
from __future__ import print_function, division
import sys, os
from RAFT.core.raft import RAFT
import tqdm

# sys.path.append('core')

import argparse
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from RAFT.core.datasets import MpiSintel, KubricDataset
from RAFT.core.utils import flow_viz
from RAFT.core.utils.utils import InputPadder
from RAFT.core.utils.flow_viz import flow_to_color
import einops

import cv2
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from matplotlib import gridspec
import scipy
import seaborn as sns
import pandas as pd
import time


os.chdir('/mnt/datagrid/personal/neoral/repos/raft_new_debug/RAFT/')

In [2]:
myhost = os.uname()[1]
print(myhost)

boruvka


In [3]:
def mkdir_if_not_exist(path):
    if not os.path.isdir(path):
        os.makedirs(path)

def mkdir_from_full_file_path_if_not_exist(path):
    basename = os.path.basename(path)
    mkdir_if_not_exist(path[:-len(basename)])

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__.update(kwargs)

In [4]:
default_kwargs = {
    'model': '/home.stud/neoramic/repos/raft_new_debug/RAFT/checkpoints/50000_raft-things-sintel-splitted-occlusion-uncertainty-base-sintel.pth',
    'save_path': '/datagrid/personal/neoral/RAFT_occl_uncertainty_outputs/statistics_occl',
    'iters': 12,
    'gpus': 0,
    'debug': True,
    'subsplit': 'validation'
}
args = AttrDict(**default_kwargs)

In [5]:
device = f'cuda:{args.gpus}'

raft_kwargs = {
    'occlusion_module': 'separate_with_uncertainty',
    'restore_ckpt': args.model,
    'small': False,
    'mixed_precision': False,
}
raft_params = AttrDict(**raft_kwargs)

# Of course, you can use argparse instead of attr_dict
raft = torch.nn.DataParallel(RAFT(raft_params))
raft.load_state_dict(torch.load(args.model, map_location='cpu'))

raft = raft.module
raft.requires_grad_(False)
raft.to(device)
raft.eval();

RAFT(
  (fnet): BasicEncoder(
    (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (relu1): ReLU(inplace=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=Fa

In [15]:
B = 1

im1_np = np.zeros((B, 3, 512, 512), np.float32)
im2_np = np.zeros((B, 3, 512, 512), np.float32)

im1 = torch.from_numpy(im1_np)
im2 = torch.from_numpy(im2_np)

padder = InputPadder(im1.shape, mode='sintel')
im1, im2 = padder.pad(im1, im2)

im1 = im1.to(device=device)
im2 = im2.to(device=device)


In [16]:
number_of_comp = 100

# warm up
for i in range(3):
    all_predictions = raft(im1.to(device), im2.to(device), iters=args.iters, test_mode=True)

st = time.time()
for i in range(number_of_comp):
    all_predictions = raft(im1.to(device), im2.to(device), iters=args.iters, test_mode=True)
en = time.time()

mean_elapse_time = (en - st) / number_of_comp
print(f'Mean elapsed time {mean_elapse_time}s, for batch size {B} with occl and sigma')

 0.5539799690246582 s


In [ ]:
B = 7

im1_np = np.zeros((B, 3, 512, 512), np.float32)
im2_np = np.zeros((B, 3, 512, 512), np.float32)

im1 = torch.from_numpy(im1_np)
im2 = torch.from_numpy(im2_np)

padder = InputPadder(im1.shape, mode='sintel')
im1, im2 = padder.pad(im1, im2)

im1 = im1.to(device=device)
im2 = im2.to(device=device)

In [13]:
number_of_comp = 100

# warm up
for i in range(3):
    all_predictions = raft(im1.to(device), im2.to(device), iters=args.iters, test_mode=True)

st = time.time()
for i in range(number_of_comp):
    all_predictions = raft(im1.to(device), im2.to(device), iters=args.iters, test_mode=True)
en = time.time()

mean_elapse_time = (en - st) / number_of_comp
print(f'Mean elapsed time {mean_elapse_time}s, for batch size {B} with occl and sigma')

0.12216700553894043


In [17]:
default_kwargs = {
    'model': '/home.stud/neoramic/repos/raft_new_debug/RAFT/models/raft-sintel.pth',
    'save_path': '/datagrid/personal/neoral/RAFT_occl_uncertainty_outputs/statistics_occl',
    'iters': 12,
    'gpus': 0,
    'debug': True,
    'subsplit': 'validation'
}
args = AttrDict(**default_kwargs)

In [20]:
device = f'cuda:{args.gpus}'

raft_kwargs = {
    'occlusion_module': None,
    'restore_ckpt': args.model,
    'small': False,
    'mixed_precision': False,
}
raft_params = AttrDict(**raft_kwargs)

# Of course, you can use argparse instead of attr_dict
raft = torch.nn.DataParallel(RAFT(raft_params))
raft.load_state_dict(torch.load(args.model, map_location='cpu'))

raft = raft.module
raft.requires_grad_(False)
raft.to(device)
raft.eval();

In [21]:
B = 1

im1_np = np.zeros((B, 3, 512, 512), np.float32)
im2_np = np.zeros((B, 3, 512, 512), np.float32)

im1 = torch.from_numpy(im1_np)
im2 = torch.from_numpy(im2_np)

padder = InputPadder(im1.shape, mode='sintel')
im1, im2 = padder.pad(im1, im2)

im1 = im1.to(device=device)
im2 = im2.to(device=device)

number_of_comp = 100

# warm up
for i in range(3):
    all_predictions = raft(im1.to(device), im2.to(device), iters=args.iters, test_mode=True)

st = time.time()
for i in range(number_of_comp):
    all_predictions = raft(im1.to(device), im2.to(device), iters=args.iters, test_mode=True)
en = time.time()

mean_elapse_time = (en - st) / number_of_comp
print(f'Mean elapsed time {mean_elapse_time}s, for batch size {B}, original raft')

Mean elapsed time 0.11844016790390015s, for batch size 1, original raft


In [22]:
B = 7

im1_np = np.zeros((B, 3, 512, 512), np.float32)
im2_np = np.zeros((B, 3, 512, 512), np.float32)

im1 = torch.from_numpy(im1_np)
im2 = torch.from_numpy(im2_np)

padder = InputPadder(im1.shape, mode='sintel')
im1, im2 = padder.pad(im1, im2)

im1 = im1.to(device=device)
im2 = im2.to(device=device)

number_of_comp = 100

# warm up
for i in range(3):
    all_predictions = raft(im1.to(device), im2.to(device), iters=args.iters, test_mode=True)

st = time.time()
for i in range(number_of_comp):
    all_predictions = raft(im1.to(device), im2.to(device), iters=args.iters, test_mode=True)
en = time.time()

mean_elapse_time = (en - st) / number_of_comp
print(f'Mean elapsed time {mean_elapse_time}s, for batch size {B}, original raft')

Mean elapsed time 0.5494323372840881s, for batch size 7, original raft
